In [20]:
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
from sklearn.cluster import KMeans
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np
nlp = spacy.load("de")

In [18]:
%run src/file_utils.py
%run src/configuration.py

In [ ]:
documents = [
    'BMW-AnnualReport-2010.json', 'BMW-AnnualReport-2011.json', 
    'BMW-AnnualReport-2012.json','BMW-AnnualReport-2013.json', 
    'BMW-AnnualReport-2014.json','BMW-AnnualReport-2015.json', 
    'BMW-AnnualReport-2016.json', 'BMW-AnnualReport-2017.json', 
 
    'CarlZeissMeditec-AnnualReport-2011.json',
    'CarlZeissMeditec-AnnualReport-2012.json', 'CarlZeissMeditec-AnnualReport-2013.json', 
    'CarlZeissMeditec-AnnualReport-2014.json','CarlZeissMeditec-AnnualReport-2015.json', 
    'CarlZeissMeditec-AnnualReport-2016.json', 'CarlZeissMeditec-AnnualReport-2017.json',
 
    'BVB-AnnualReport-2010.json', 'BVB-AnnualReport-2011.json', 
    'BVB-AnnualReport-2012.json', 'BVB-AnnualReport-2013.json', 
    'BVB-AnnualReport-2014.json', 'BVB-AnnualReport-2015.json',
    'BVB-AnnualReport-2016.json', 'BVB-AnnualReport-2017.json']

In [25]:
TYPE = 'type'
PARAGRAPH = 'paragraph'
CONTENT = 'content'

In [16]:
def readContentOfFile(file_name):
    content = ''
    try:
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    except:
        FileUtils.fix_json(file_name)
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    return content

In [108]:
extra_stop = ['million', 'tausend', 'eur', 'teur', '*', '+', '&','%']
def perform_lemmatization(document):
    content_of_document = readContentOfFile(document)[0:999999]
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutcurrency = [word for word in filtered_words_withoutdigits if not word.is_currency]
    filtered_words_withoutverbs = [word for word in filtered_words_withoutcurrency if word.pos_ != 'VERB']
    filtered_words_withoutnum = [word for word in filtered_words_withoutverbs if word.pos_ != 'NUM']
    filtered_words_withoutsym = [word for word in filtered_words_withoutnum if word.pos_ != 'SYM']
    filtered_words_withoutpunc = [word for word in filtered_words_withoutsym if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    filtered_lemmas = [word for word in filtered_lemmas if not word in extra_stop ]
    lemmatized_content = " ".join(item for item in filtered_lemmas)
    return lemmatized_content.lower()

In [9]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(FILE_PATH) if isfile(join(FILE_PATH, f))]
onlyfiles = onlyfiles[0:200]

# Lemmatization of documents

In [174]:
start_time = time.time()
lemm_docs_prep = [ perform_lemmatization(FILE_PATH + document) for document in onlyfiles]
print (time.time() - start_time)

313.35024881362915


# Construct vocabulary for BMW

In [244]:
bmw_lemm_docs_prep = [doc.replace("\n", "") for doc in bmw_lemm_docs_prep]

In [146]:
bmw_lemm_docs_prep = [
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2010.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2011.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2012.json'),
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2013.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2014.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2015.json'),
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2016.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2017.json')]

In [21]:
def clean_text(text):
    lines = []
    for line in text.splitlines():
        if line.endswith('-\n'):
            line.replace('-\n', '')
        else:
            line.replace('\n', ' \n')
        lines.append(line)
    return ''.join(lines)

In [24]:
def print_with_rep_top_words(model, feature_names, n_top_words):
    matr = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        
        message += " ".join([str(feature_names[i]) + ": " + "{:.5f}".format(matr[topic_idx, i])
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        print()
    print()

In [284]:
bmw_files = [f for f in listdir(FILE_PATH) if isfile(join(FILE_PATH, f)) and 'BMW' in f]
bmw_files
banks

['Landesbank_Baden-Wuerttemberg-QuarterlyReport-2017-Q1.json',
 'DeutschePfandbriefbank-QuarterlyReport-2016-Q2.json',
 'BayerischeLandesbank-QuarterlyReport-2012-Q2.json',
 'DeutschePfandbriefbank-AnnualReport-2011.json',
 'BayerischeLandesbank-QuarterlyReport-2010-Q2.json',
 'DeutschePfandbriefbank-QuarterlyReport-2015-Q3.json',
 'DzBank-AnnualReport-2015.json',
 'Landesbank_Hessen-Thueringen-AnnualReport-2010.json',
 'Commerzbank-QuarterlyReport-2012-Q1.json',
 'NorddeutscheLandesbank-QuarterlyReport-2017-Q1.json',
 'Commerzbank-QuarterlyReport-2012-Q3.json',
 'Landesbank_Baden-Wuerttemberg-AnnualReport-2011.json',
 'Landesbank_Hessen-Thueringen-QuarterlyReport-2013-Q2.json',
 'Commerzbank-QuarterlyReport-2010-Q2.json',
 'BayerischeLandesbank-QuarterlyReport-2014-Q3.json',
 'NorddeutscheLandesbank-QuarterlyReport-2016-Q3.json',
 'DzBank-AnnualReport-2010.json',
 'Landesbank_Baden-Wuerttemberg-QuarterlyReport-2011-Q2.json',
 'Commerzbank-QuarterlyReport-2013-Q2.json',
 'DzBank-Quarte

In [71]:
company_names = set()
files = [f for f in listdir(FILE_PATH) if isfile(join(FILE_PATH, f))]
for file in files:
    company_names.add(file[:file.find('-')])
company_names

{'ADOProperties',
 'ADVA_Optical',
 'Aareal',
 'Adidas',
 'AdlerRealEstate',
 'Airbus',
 'Aixtron',
 'Allianz',
 'AmadeusFiRe',
 'Aurubis',
 'Axel_Springer',
 'BASF',
 'BMW',
 'BayWa',
 'Bayer',
 'BayerischeLandesbank',
 'Bechtle',
 'Beiersdorf',
 'Bertrandt',
 'BetAtHome',
 'Bilfinger',
 'Biotest',
 'Bosch',
 'Brenntag',
 'alstria'}

In [104]:
company_names = set()
banks = [f for f in listdir(FILE_PATH) if isfile(join(FILE_PATH, f)) and 'bank' in f.lower()]

In [21]:
len(banks)

176

In [109]:
bmw_lemm_docs_prep = [clean_text(perform_lemmatization(FILE_PATH + file)) for file in banks]

In [307]:
vectorizer_bmw = TfidfVectorizer()
start_time = time.time()
tfidf_matrix_bmw = vectorizer_bmw.fit_transform(bmw_lemm_docs_prep)
print (time.time() - start_time)

0.2548213005065918


In [308]:
bmw_feature_names = vectorizer_bmw.get_feature_names()
bmw_corpus_index = [n for n in [
    'BMW-2010', 'BMW-2011', 'BMW-2012', 
    'BMW-2013', 'BMW-2014', 'BMW-2015',
    'BMW-2016', 'BMW-2017']]
idf = vectorizer_bmw.idf_
df = pd.DataFrame(tfidf_matrix_bmw.T.todense(), index=bmw_feature_names)
df['idf'] = idf

In [309]:
#df = df.sort_values(by=['BMW-2010'], ascending=False)
bmw_df = df[(df['idf'] != 1)]

In [310]:
bmw_vocab = set()
bmw_vocab.update(bmw_df.index.tolist())

In [22]:
lem_pars = []
for file in banks:
    lem_pars += lemmatize_paragraphs(readContentOfParagraphs(FILE_PATH + file))

In [96]:
commerz_par = lemmatize_paragraphs(readContentOfParagraphs(FILE_PATH + 'Commerzbank-AnnualReport-2016.json'))

In [219]:
lem_pars[0]

'banken aufgrund anforderung gemäß crr capital requirements regulation – verordnung eu nummer crd iv capital requirements directive iv –richtlinie verpflichten mindestens jährlich offenlegungsbericht erstellen'

In [25]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(lem_pars)

lda = LatentDirichletAllocation(n_components=9,
                                learning_method='batch',
                                random_state=0)
lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_with_rep_top_words(lda, tf_feature_names, 10)

Topic #0: bank: 0.02279 dzbank: 0.01210 gruppe: 0.01032 dz: 0.01031 aufsichtsrat: 0.00979 unternehmen: 0.00780 vorstehen: 0.00711 mitglied: 0.00703 vorstand: 0.00565 aufsichtsrats: 0.00502

Topic #1: unternehmen: 0.01723 aktie: 0.01609 anteil: 0.01347 bank: 0.01067 vergütung: 0.00956 konzern: 0.00945 equity: 0.00844 gesellschaft: 0.00803 tochterunternehmen: 0.00600 commerzbank: 0.00572

Topic #2: million: 0.05586 euro: 0.04462 milliarde: 0.02668 vorjahr: 0.01818 höhe: 0.01500 dezember: 0.01380 ergebnis: 0.01191 eur: 0.01099 mrd: 0.01040 betragen: 0.00877

Topic #3: mio: 0.03075 segment: 0.01063 höhe: 0.01038 rückstellung: 0.00932 konzern: 0.00782 aufwendung: 0.00718 vermögenswerte: 0.00715 enthalten: 0.00632 ertrag: 0.00597 betragen: 0.00569

Topic #4: konzern: 0.00962 sicherheit: 0.00950 kredit: 0.00815 us: 0.00809 amerikanischen: 0.00802 forderung: 0.00783 vermögenswerte: 0.00765 verbindlichkeit: 0.00752 wertpapiere: 0.00672 mrd: 0.00603

Topic #5: bank: 0.01383 kunde: 0.01090 deutsc

In [114]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_with_rep_top_words(lda, tf_feature_names, 1000)

Topic #0: bank: 0.02279 dzbank: 0.01210 gruppe: 0.01032 dz: 0.01031 aufsichtsrat: 0.00979 unternehmen: 0.00780 vorstehen: 0.00711 mitglied: 0.00703 vorstand: 0.00565 aufsichtsrats: 0.00502 sitzung: 0.00414 person: 0.00387 mein: 0.00378 deutsche: 0.00346 corporate: 0.00331 geschäftsjahr: 0.00326 geschäft: 0.00312 insbesondere: 0.00305 deutschland: 0.00293 volksbanken: 0.00292 ratings: 0.00283 rating: 0.00269 raiffeisenbanken: 0.00268 commerzbank: 0.00257 stellen: 0.00257 zudem: 0.00254 berichten: 0.00248 governance: 0.00245 lbbw: 0.00243 kunde: 0.00243 hauptversammlung: 0.00240 engagement: 0.00238 aktuell: 0.00236 finanzgruppe: 0.00235 stehen: 0.00234 hinaus: 0.00232 wesentlich: 0.00222 prüfung: 0.00211 nahestehend: 0.00210 zielen: 0.00209 nehmen: 0.00205 prüfungsausschuss: 0.00202 investment: 0.00199 mitarbeiter: 0.00197 regelmäßig: 0.00196 thema: 0.00195 rv: 0.00194 deutschen: 0.00192 sektor: 0.00188 vorsitzende: 0.00187 rahmen: 0.00186 ausschuß: 0.00186 unterstützen: 0.00182 herr: 0.

In [97]:
tf_whole = tf_vectorizer.transform(commerz_par)
topic_model = lda.transform(tf_whole)

In [98]:
print()
print(' Dominant topics per document ')
print('------------------------------')


for doc, document_name in enumerate(range(len(commerz_par))):
    print('\n{:40.40}: '.format(str(document_name)), end ='')
    most_probable = np.argsort(topic_model[doc, :])[:-6:-1]

    cummulated = 0
    for topic in most_probable:
        probability = topic_model[doc, topic]
        print('{:6.2%} {:3} '.format(probability, topic), end = '')
        cummulated = cummulated + probability
        if cummulated > 0.95: break


 Dominant topics per document 
------------------------------

0                                       : 50.23%   5 46.99%   7 
1                                       : 85.68%   8 13.30%   5 
2                                       : 28.86%   0 24.86%   1 23.00%   3 16.69%   5  5.49%   8 
3                                       : 45.56%   0 29.01%   6 24.91%   5 
4                                       : 37.21%   2 34.15%   5 20.08%   6  8.35%   0 
5                                       : 30.83%   5 22.18%   7 20.89%   0 19.32%   6  6.58%   2 
6                                       : 96.84%   0 
7                                       : 91.99%   0  6.60%   4 
8                                       : 76.01%   0 17.22%   4  3.40%   5 
9                                       : 89.00%   0  9.24%   4 
10                                      : 91.94%   0  4.74%   1 
11                                      : 82.67%   0 11.03%   7  3.02%   4 
12                                      : 88.7

634                                     : 79.78%   1 13.15%   2  1.01%   4  1.01%   3  1.01%   8 
635                                     : 64.16%   1 31.74%   2 
636                                     : 93.65%   1  0.79%   8  0.79%   7 
637                                     : 56.22%   1 40.07%   2 
638                                     : 81.68%   1 13.74%   2 
639                                     : 88.71%   1  8.78%   8 
640                                     : 89.28%   8  7.18%   2 
641                                     : 85.76%   8  8.05%   1  5.04%   2 
642                                     : 70.79%   1 18.10%   8 10.20%   2 
643                                     : 95.32%   1 
644                                     : 94.44%   1  0.70%   6 
645                                     : 98.29%   1 
646                                     : 98.41%   1 
647                                     : 97.78%   1 
648                                     : 80.32%   1 17.33%   4 
649

In [99]:
# for paragraphs
print()
print(' Dominant topics per document ')
print('------------------------------')

bank_risks = []
for doc, document_name in enumerate(range(len(commerz_par))):
    company_name = document_name
    

        
    
    print('\n{:40.40}: '.format(str(document_name)), end ='')
    most_probable = np.argsort(topic_model[doc, :])[:-6:-1]

    cummulated = 0
    for topic in most_probable:

        probability = topic_model[doc, topic]
        if int(topic) == 7:
            bank_risks.append({
                'paragraph': company_name,
                'value': str(probability)
            })
        print('{:6.2%} {:3} '.format(probability, topic), end = '')
        cummulated = cummulated + probability
        if cummulated > 0.95: break


 Dominant topics per document 
------------------------------

0                                       : 50.23%   5 46.99%   7 
1                                       : 85.68%   8 13.30%   5 
2                                       : 28.86%   0 24.86%   1 23.00%   3 16.69%   5  5.49%   8 
3                                       : 45.56%   0 29.01%   6 24.91%   5 
4                                       : 37.21%   2 34.15%   5 20.08%   6  8.35%   0 
5                                       : 30.83%   5 22.18%   7 20.89%   0 19.32%   6  6.58%   2 
6                                       : 96.84%   0 
7                                       : 91.99%   0  6.60%   4 
8                                       : 76.01%   0 17.22%   4  3.40%   5 
9                                       : 89.00%   0  9.24%   4 
10                                      : 91.94%   0  4.74%   1 
11                                      : 82.67%   0 11.03%   7  3.02%   4 
12                                      : 88.7

414                                     : 66.80%   7 31.15%   2 
415                                     : 93.65%   1  0.79%   2  0.79%   8 
416                                     : 93.65%   1  0.79%   2  0.79%   8 
417                                     : 93.65%   1  0.79%   2  0.79%   8 
418                                     : 65.98%   8 28.04%   1  0.86%   3  0.86%   2 
419                                     : 59.19%   1 32.40%   2  6.89%   3 
420                                     : 66.81%   1 30.76%   2 
421                                     : 69.52%   8 30.01%   5 
422                                     : 56.49%   8 33.71%   7  9.44%   3 
423                                     : 61.43%   8 16.20%   3 13.38%   6  8.70%   5 
424                                     : 56.88%   8 23.46%   6 18.42%   1 
425                                     : 62.39%   8 22.13%   6 13.52%   1 
426                                     : 40.92%   8 29.52%   6 27.18%   2 
427                    

736                                     : 63.83%   2 30.61%   0  0.79%   3 
737                                     : 79.69%   0 14.32%   3  0.86%   1  0.86%   6 
738                                     : 54.05%   0 16.22%   4 15.31%   2 11.64%   3 
739                                     : 73.38%   1 22.73%   0 
740                                     : 40.63%   4 35.17%   0 19.07%   2  0.86%   1 
741                                     : 63.78%   1 31.36%   0 
742                                     : 81.73%   1 16.05%   3 
743                                     : 67.99%   8 28.77%   1 
744                                     : 68.68%   1 14.15%   8 11.45%   6  5.10%   2 
745                                     : 61.62%   1 18.77%   2 14.85%   3 
746                                     : 57.47%   2 31.40%   1  6.97%   7 
747                                     : 50.07%   1 21.16%   0 18.11%   2  9.95%   5 
748                                     : 92.59%   1  0.93%   0  0.93%   2  0

In [105]:
bank_risks
newlist = sorted(bank_risks, key=lambda k: -1 * float(k['value']), )
top_result = [paragraph for paragraph in newlist if float(paragraph['value']) > 0.5]
value_sum = 0.0
top_result_page_number = []
for result in top_result:
    item = {}
    value_sum += float(result['value'])
    item['paragraph'] = result['paragraph']
    item['value'] = result['value']
    item['page'] = commerz_par_no_lem_num[result['paragraph']]['page']
    item['page_par'] = commerz_par_no_lem_num[result['paragraph']]['paragraph']
    top_result_page_number.append(item)
top_result_page_number
top_result_page_number = sorted(top_result_page_number, key=lambda k:int(k['page']) * 100 + int(k['page_par']))
top_result_page_number

[{'paragraph': 60, 'value': '0.6747296307066113', 'page': 26, 'page_par': 4},
 {'paragraph': 96, 'value': '0.8243989767108063', 'page': 43, 'page_par': 1},
 {'paragraph': 98, 'value': '0.5600557850651292', 'page': 43, 'page_par': 3},
 {'paragraph': 99, 'value': '0.7177469533290974', 'page': 43, 'page_par': 4},
 {'paragraph': 100, 'value': '0.5055973074987608', 'page': 43, 'page_par': 5},
 {'paragraph': 103, 'value': '0.8268694117481222', 'page': 44, 'page_par': 1},
 {'paragraph': 104, 'value': '0.5420705537382244', 'page': 44, 'page_par': 2},
 {'paragraph': 105, 'value': '0.7634071341664692', 'page': 44, 'page_par': 3},
 {'paragraph': 106, 'value': '0.5752526285515214', 'page': 44, 'page_par': 4},
 {'paragraph': 107, 'value': '0.7791469386034343', 'page': 45, 'page_par': 0},
 {'paragraph': 111, 'value': '0.9473255541123967', 'page': 45, 'page_par': 4},
 {'paragraph': 114, 'value': '0.7040795162409503', 'page': 46, 'page_par': 2},
 {'paragraph': 118, 'value': '0.5479961118090952', 'page

In [106]:
from itertools import groupby

schedule = dict()
for k, v in groupby(top_result_page_number, lambda x: x['page']):
    schedule[k] = list(v)

In [107]:
values = []
pages = []
texts = []
for k,v in schedule.items():
#    val_sum = 0.0
#    for item in v:
#        val_sum += float(item['value'])
    value = max([item['value'] for item in v])
    values.append(value)
    pages.append(k)
    max_par = max([item['paragraph'] for item in v])
    texts.append(str(k) + '\n' + str(max_par))


In [108]:
import plotly as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)


trace0 = go.Bar(
    x=pages,
    y=values,
    text=texts
)

data = [trace0]
layout = go.Layout(
    title='Commerzbank 2016 Report',
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename='text-hover-bar')

In [113]:
commerz_par_no_lem[387]

'In der Commerzbank sind die Themenstellungen OpRisk und\nGovernance des Internen Kontrollsystems (IKS) aufbauorganisato-\nrisch und methodisch eng miteinander verbunden und werden\nkontinuierlich weiterentwickelt. Hintergrund dieser Verzahnung\nist, dass in zahlreichen OpRisk-Fällen die Ursachen im Zusam-\nmenhang mit dem Versagen der Kontrollmechanismen stehen.\nDamit trägt ein funktionsfähiges IKS dazu bei, Verluste aus opera-\ntionellen Risiken zu reduzieren beziehungsweise zu vermeiden\nund so mittel- bis langfristig auch die Kapitalbelastung für die Hin-\nterlegung Operationeller Risiken zu mindern. Umgekehrt bieten\ndie Systematiken für operationelle Risiken die Möglichkeit, das\nIKS konsistent zum Management operationeller Risiken auszu-\nrichten. Die Stärkung des IKS-Kontrollgefüges ist eine wesentliche\nSäule zur proaktiven Reduzierung beziehungsweise Verhinderung\noperationeller Risiken.\nDas Group OpRisk Committee der Commerzbank tagt vier Mal\njährlich unter Vorsitz des Ch

In [34]:
def readPageAndParInfo(file_name):
    contents = []
    try:
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    contents.append({
                        'page':item['pagenumber'],
                        'paragraph':item['counter']
                    })
    except:
        FileUtils.fix_json(file_name)
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    contents.append({
                        'page':item['pagenumber'],
                        'paragraph':item['counter']
                    })
    return contents

In [95]:
commerz_par_no_lem = readContentOfParagraphs(FILE_PATH + 'Commerzbank-AnnualReport-2016.json')
commerz_par_no_lem_num = readPageAndParInfo(FILE_PATH + 'Commerzbank-AnnualReport-2016.json')

In [281]:
print()
print(' Dominant topics per document ')
print('------------------------------')

bank_risks = {}
for doc, document_name in enumerate([file for file in banks]):
    if 'Annual' not in document_name:
        continue
    company_name = document_name[:document_name.find('-')]
    
    if company_name not in bank_risks:
        bank_risks[company_name] = []
        
    
    print('\n{:40.40}: '.format(document_name), end ='')
    most_probable = np.argsort(topic_model[doc, :])[:-6:-1]

    cummulated = 0
    for topic in most_probable:

        probability = topic_model[doc, topic]
        if int(topic) == 5:
            year = document_name[document_name.rfind('-') + 1:document_name.rfind('-') + 5]
            bank_risks[company_name].append({
                'year': year,
                'value': str(probability)
            })
        print('{:6.2%} {:3} '.format(probability, topic), end = '')
        cummulated = cummulated + probability
        if cummulated > 0.95: break


 Dominant topics per document 
------------------------------

DeutschePfandbriefbank-AnnualReport-2011: 19.86%   8 17.26%   7 16.71%   5 15.24%   2  9.05%   0 
DzBank-AnnualReport-2015.json           : 21.66%   0 18.93%   7 12.82%   2 12.26%   8  8.17%   1 
Landesbank_Hessen-Thueringen-AnnualRepor: 24.87%   5 16.07%   7 11.65%   8  9.22%   2  8.75%   0 
Landesbank_Baden-Wuerttemberg-AnnualRepo: 19.51%   8 15.45%   7 13.48%   0 13.30%   2 10.60%   5 
DzBank-AnnualReport-2010.json           : 28.12%   0 13.63%   7 12.25%   3 12.19%   8 10.41%   2 
BayerischeLandesbank-AnnualReport-2012.j: 27.47%   7 24.21%   2 10.31%   0  8.90%   5  7.34%   6 
BayerischeLandesbank-AnnualReport-2016.j: 23.83%   2 20.74%   7 16.52%   8 10.33%   5  7.38%   1 
DzBank-AnnualReport-2012.json           : 27.17%   0 14.29%   7 13.07%   8  9.31%   2  8.26%   6 
Landesbank_Baden-Wuerttemberg-AnnualRepo: 20.00%   8 16.71%   2 14.47%   7 12.02%   0  9.10%   1 
Commerzbank-AnnualReport-2011.json      : 21.17%   2 1

In [282]:
bank_risks['Commerzbank']

[{'year': '2011', 'value': '0.1499289298804297'},
 {'year': '2014', 'value': '0.15728062086046277'},
 {'year': '2016', 'value': '0.13638580327531857'},
 {'year': '2013', 'value': '0.1376108640937372'},
 {'year': '2010', 'value': '0.16478086962300859'},
 {'year': '2015', 'value': '0.14615132421070198'},
 {'year': '2012', 'value': '0.13034352706350055'}]

In [283]:
values = []
for key, value in bank_risks.items():
    newlist = sorted(bank_risks[key], key=lambda k: int(k['year']))
    for l in newlist:
        values.append(round(float(l['value']) * 100,2))
    print (key)    
    print (values)
    values = []

DeutschePfandbriefbank
[20.06, 16.71, 14.45, 14.31, 13.48, 10.98]
DzBank
[]
Landesbank_Hessen
[24.87, 25.84, 26.38, 14.08, 19.46, 20.37, 12.02]
Landesbank_Baden
[12.25, 10.6, 10.05, 8.87, 9.17]
BayerischeLandesbank
[8.09, 7.99, 8.9, 6.88, 9.7, 10.32, 10.33]
Commerzbank
[16.48, 14.99, 13.03, 13.76, 15.73, 14.62, 13.64]
NorddeutscheLandesbank
[16.28, 18.23, 17.38, 15.95]
DeutscheBank
[9.55, 10.29]


In [144]:
m_file = 'BayerischeLandesbank-AnnualReport-2012.json'
company_name = m_file[:m_file.find('-')]
print(company_name)
year = m_file[m_file.rfind('-') + 1:m_file.rfind('-') + 5]
print(year)

BayerischeLandesbank
2012


In [27]:
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(bmw_lemm_docs_prep)

lda = LatentDirichletAllocation(n_components=7,
                                learning_method='batch')
topic_model = lda.fit_transform(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_with_rep_top_words(lda, tf_feature_names, 5)

Topic #0: verdienst: 0.00006 entwicklungsarbeit: 0.00006 bleivergangenen: 0.00006 kundenreaktionen: 0.00006 marktrisiko: 0.00006
Topic #1: bmw: 0.02569 million: 0.01526 group: 0.01274 unternehmen: 0.00514 höhe: 0.00407
Topic #2: verdienst: 0.00006 entwicklungsarbeit: 0.00006 bleivergangenen: 0.00006 kundenreaktionen: 0.00006 marktrisiko: 0.00006
Topic #3: verdienst: 0.00006 entwicklungsarbeit: 0.00006 bleivergangenen: 0.00006 kundenreaktionen: 0.00006 marktrisiko: 0.00006
Topic #4: verdienst: 0.00006 entwicklungsarbeit: 0.00006 bleivergangenen: 0.00006 kundenreaktionen: 0.00006 marktrisiko: 0.00006
Topic #5: bmw: 0.02756 group: 0.01582 million: 0.01088 unternehmen: 0.00567 risiko: 0.00469
Topic #6: verdienst: 0.00006 entwicklungsarbeit: 0.00006 bleivergangenen: 0.00006 kundenreaktionen: 0.00006 marktrisiko: 0.00006



In [317]:
tf

<8x15515 sparse matrix of type '<class 'numpy.int64'>'
	with 38640 stored elements in Compressed Sparse Row format>

In [316]:
print()
print(' Dominant topics per document ')
print('------------------------------')


for doc, document_name in enumerate([file for file in bmw_files if 'Annual' in file]):
    
    print('\n{:40.40}: '.format(document_name), end ='')
    most_probable = np.argsort(topic_model[doc, :])[:-6:-1]

    cummulated = 0
    for topic in most_probable:
        probability = topic_model[doc, topic]
        print('{:6.2%} {:3} '.format(probability, topic), end = '')
        cummulated = cummulated + probability
        if cummulated > 0.95: break


 Dominant topics per document 
------------------------------

BMW-AnnualReport-2017.json              : 99.99%   2 
BMW-AnnualReport-2013.json              : 99.99%   2 
BMW-AnnualReport-2011.json              : 99.99%   5 
BMW-AnnualReport-2014.json              : 99.99%   2 
BMW-AnnualReport-2016.json              : 99.99%   2 
BMW-AnnualReport-2012.json              : 99.99%   5 
BMW-AnnualReport-2015.json              : 66.88%   2 33.11%   5 
BMW-AnnualReport-2010.json              : 99.99%   5 

# LDA for one document (analyze paragraphs)

In [28]:
def readContentOfParagraphs(file_name):
    contents = []
    try:
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    contents.append(item[CONTENT])
    except:
        FileUtils.fix_json(file_name)
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    contents.append(item[CONTENT])
    return contents

In [29]:
nlp = spacy.load("de")
def perform_lemmatization(document):
    content_of_document = readContentOfFile(document)
    
    #remove minus sign and next-line sign
    content_of_document = content_of_document.replace('-\n','')
    content_of_document = content_of_document.replace('\n',' ')
    
    #replace all gco2 with co2
    content_of_document = content_of_document.replace('gCO2','CO2')
    
    #remove the character we don't need
    remove_char = content_of_document.maketrans('-',' ','+*<>%/&$')
    content_of_document = content_of_document.translate(remove_char)
    
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    
    final = []  
    for item in filtered_lemmas:
        #remove the words contain digit except of co2
        if(any(c.isdigit() for c in item)):
            if 'CO2' in item:
                final.append(item)
        else:
            #remove the words contain dot
            if '.' not in item:
                final.append(item)
    
    lemmatized_content = " ".join(item for item in final)
    
    #output the result into file 
    #if document.startswith(FILE_PATH):
    #    filename = "filtered_" + document[len(FILE_PATH):]
    
    #with open(filename, 'w') as outfile:
    #    json.dump(lemmatized_content.lower(), outfile)
    
    return lemmatized_content.lower()

In [30]:
def lemmatize_paragraphs(paragraphs):
    lemmatized_paragraphs = []
    for paragraph in paragraphs:
        content_of_document = paragraph.replace('-\n','')
        content_of_document = content_of_document.replace('\n',' ')
    
        #replace all gco2 with co2
        content_of_document = content_of_document.replace('gCO2','CO2')
    
        #remove the character we don't need
        remove_char = content_of_document.maketrans('-',' ','+*<>%/&$')
        content_of_document = content_of_document.translate(remove_char)
    
        sentence = nlp(content_of_document)
        filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
        filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
        filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
        filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    
        final = []  
        for item in filtered_lemmas:
            #remove the words contain digit except of co2
            
            if(any(c.isdigit() for c in item)):
                if 'CO2' in item:
                    final.append(item)
            else:
                #remove the words contain dot
                if '.' not in item:
                    final.append(item)
    
        lemmatized_content = " ".join(item for item in final)
        lemmatized_paragraphs.append(lemmatized_content.lower())
    return lemmatized_paragraphs

In [7]:
lem_pars = lemmatize_paragraphs(readContentOfParagraphs(FILE_PATH + 'BMW-AnnualReport-2010.json'))
lem_pars = lemmatize_paragraphs(readContentOfParagraphs(FILE_PATH + 'BMW-AnnualReport-2016.json'))

PermissionError: [Errno 13] Permission denied: './LabShare/data/all/json/BMW-AnnualReport-2010.json'

In [433]:
vectorizer_bmw = TfidfVectorizer()
start_time = time.time()
tfidf_matrix_bmw = vectorizer_bmw.fit_transform(lem_pars)
print (time.time() - start_time)

0.057863712310791016


In [434]:
bmw_feature_names = vectorizer_bmw.get_feature_names()
bmw_corpus_index = [n for n in [
    'BMW-2010', 'BMW-2011', 'BMW-2012', 
    'BMW-2013', 'BMW-2014', 'BMW-2015',
    'BMW-2016', 'BMW-2017']]
idf = vectorizer_bmw.idf_
df = pd.DataFrame(tfidf_matrix_bmw.T.todense(), index=bmw_feature_names)
df['idf'] = idf

In [435]:
bmw_df = df[(df['idf'] != 1)]

In [436]:
bmw_vocab = set()
bmw_vocab.update(bmw_df.index.tolist())

In [518]:
tf_vectorizer = CountVectorizer(stop_words=[' ', 'group'])
tf = tf_vectorizer.fit_transform(lem_pars)

lda = LatentDirichletAllocation(n_components=10,
                                learning_method='batch')
lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_with_rep_top_words(lda, tf_feature_names, 10)

Topic #0: mitglied: 0.01955 aufsichtsrat: 0.01605 vorstand: 0.01597 aufsichtsrats: 0.01035 vorstehen: 0.01015 unternehmen: 0.00664 corporate: 0.00522 governance: 0.00502 insbesondere: 0.00456 vergütung: 0.00450
Topic #1: segment: 0.00955 automobile: 0.00702 unternehmen: 0.00672 geschäftsjahr: 0.00601 kunde: 0.00589 ergebnis: 0.00558 berücksichtigen: 0.00427 steuer: 0.00382 steuerung: 0.00374 liegen: 0.00372
Topic #2: compliance: 0.01608 fahrzeug: 0.00913 motorrad: 0.00709 next: 0.00709 roce: 0.00561 strategie: 0.00558 one: 0.00536 number: 0.00536 segment: 0.00520 committee: 0.00498
Topic #3: sitzung: 0.01052 unternehmen: 0.00942 vorstehen: 0.00940 mitglied: 0.00724 konzernabschluss: 0.00683 vorstand: 0.00609 aufsichtsrats: 0.00594 hinaus: 0.00419 wesentlich: 0.00417 joint: 0.00415
Topic #4: risiko: 0.01135 gesellschaft: 0.00537 segment: 0.00506 entwicklung: 0.00427 information: 0.00397 erfasst: 0.00394 jeweilig: 0.00387 finanzdienstleistungen: 0.00350 amerikanischen: 0.00349 us: 0.0034

In [519]:
get_tf = tf_vectorizer.transform([bmw_lemm_docs_prep[7]])
#gen_tf_vect = CountVectorizer(vocabulary=tf_feature_names)

#get_tf = gen_tf_vect.fit_transform([bmw_lemm_docs_prep[6]])
topic_model = lda.transform(get_tf) 

In [520]:
print()
print(' Dominant topics per document ')
print('------------------------------')


for doc, document_name in enumerate(range(0,3,1)):
    
    print('\n{:40.40}: '.format(str(document_name)), end ='')
    most_probable = np.argsort(topic_model[doc, :])[:-11:-1]

    cummulated = 0
    for topic in most_probable:
        probability = topic_model[doc, topic]
        print('{:6.2%} {:3} '.format(probability, topic), end = '')
        cummulated = cummulated + probability
        if cummulated > 0.95: break


 Dominant topics per document 
------------------------------

0                                       : 16.45%   9 15.29%   3 14.94%   0  9.84%   7  9.46%   8  8.46%   4  7.64%   1  6.58%   2  6.32%   6 
1                                       : 

IndexError: index 1 is out of bounds for axis 0 with size 1

# Construct vocabulary

In [8]:
start_time = time.time()
common_vocabularly_lem = set()
for document in onlyfiles:
    content_of_document = readContentOfFile(FILE_PATH + document)
    sentence = nlp(content_of_document)
    # problems with non-lower case stop words
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutcurrency = [word for word in filtered_words_withoutdigits if not word.is_currency]
    filtered_words_withoutverbs = [word for word in filtered_words_withoutcurrency if word.pos_ != 'VERB']
    filtered_words_withoutnum = [word for word in filtered_words_withoutverbs if word.pos_ != 'NUM']
    filtered_words_withoutsym = [word for word in filtered_words_withoutnum if word.pos_ != 'SYM']
    filtered_words_withoutpunc = [word for word in filtered_words_withoutsym if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    vocabularly = set()
    for word in filtered_lemmas:
        vocabularly.add(word.replace('\n', '').strip().lower())
    new_vocab = set()
    for u in vocabularly:
        if u != '':
            new_vocab.add(u)

#     lemmatized_content = " ".join(item for item in filtered_lemmas)
#     vectorizer = TfidfVectorizer(vocabulary=new_vocab)
#     tfidf_matrix = vectorizer.fit_transform([lemmatized_content])
#     feature_names = vectorizer.get_feature_names()
#     corpus_index = [n for n in ['Values']]
#     df = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
#     df = df.sort_values(by=['Values'], ascending=False)
#     print (df.head(5).index.values.tolist())
#     common_vocabularly_lem.update(df.head(1000).index.values.tolist())
    common_vocabularly_lem.update(new_vocab)
# not removed by spacy.    
common_vocabularly_lem.remove('million')
common_vocabularly_lem.remove('tausend')
common_vocabularly_lem.remove('eur')
common_vocabularly_lem.remove('teur')
common_vocabularly_lem.remove('*')
common_vocabularly_lem.remove('+')
common_vocabularly_lem.remove('&')
common_vocabularly_lem.remove('%')
print (time.time() - start_time)

NameError: name 'onlyfiles' is not defined

In [176]:
def print_top_words(model, feature_names, n_top_words):
    glob_set = set()
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        l = []
        #print(glob_set)
        for i in topic.argsort()[:-100 - 1:-1]:
            if len(l) == n_top_words:
                break
            if feature_names[i] not in glob_set:
                glob_set.add(feature_names[i])
                l.append(feature_names[i])
        message += " ".join(l)
        print(message)
    print()

In [77]:
#common_vocabularly_lem.remove('bmw')
#common_vocabularly_lem.remove('group')
#common_vocabularly_lem.remove('carl')
#common_vocabularly_lem.remove('zeiss')
#common_vocabularly_lem.remove('borussia')
#common_vocabularly_lem.remove('dortmund')

In [192]:
tf_vectorizer = CountVectorizer(vocabulary=common_vocabularly_lem)
tf = tf_vectorizer.fit_transform(lemm_docs_prep)

In [231]:
lda = LatentDirichletAllocation(n_components=25,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=0,
                                   max_iter=50)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=25, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [232]:
tf_feature_names = tf_vectorizer.get_feature_names()
print (print_top_words(lda, tf_feature_names, 10))

Topic #0: � fraport-spezifischer fremd- fremdanteile fremdanteilen fremdbesitz fremdbezug fremdbezugsteile fremddarlehen fremddruck-
Topic #1: evt bionamics innovate euprotec execute earn janssen gegenleistung hyperion vergleichsperiode
Topic #2: fremddruckleistungen fremdeinwirkungen fremdfertigungsquote fremdfi fremdfinan- fremdfinanzie- fremdfinanzierung fremdfinanzierungen fremdfinanzierungs- fremdfinanzierungsangebot
Topic #3: bayer vorjahr sondereinflüssen umsatz cropscience usa ebitda healthcare ebit wpb
Topic #4: fremdfinanzierungsanteil fremdfinanzierungsbasis fremdfinanzierungsbedarf fremd frem- freizügigkeitsleis- freital freistaat freiste- freistehend
Topic #5: aktie euro unternehmen bilfinger gesellschaft höhe konzern aufsichtsrat prozent vorstehen
Topic #6: freistehende freistellung freistellungs- freistellungserklärungen freistellungsphase freitag freiverkehr freizeitphase freiwerdende freiwillig
Topic #7: evonik aktivität bereinigen fortgeführt ergebnis operativ vivawes

In [234]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_with_rep_top_words(lda, tf_feature_names, 10)

Topic #0: �: 0.04000 fraport-spezifischer: 0.04000 fremd-: 0.04000 fremdanteile: 0.04000 fremdanteilen: 0.04000 fremdbesitz: 0.04000 fremdbezug: 0.04000 fremdbezugsteile: 0.04000 fremddarlehen: 0.04000 fremddruck-: 0.04000
Topic #1: evt: 30.58348 bionamics: 21.96573 innovate: 21.10578 euprotec: 20.04000 execute: 19.33106 earn: 16.09364 janssen: 11.01888 gegenleistung: 10.94376 hyperion: 9.04000 vergleichsperiode: 9.03042
Topic #2: �: 0.04000 fraport-spezifischer: 0.04000 fremd-: 0.04000 fremdanteile: 0.04000 fremdanteilen: 0.04000 fremdbesitz: 0.04000 fremdbezug: 0.04000 fremdbezugsteile: 0.04000 fremddarlehen: 0.04000 fremddruck-: 0.04000
Topic #3: bayer: 242.47646 vorjahr: 136.19577 sondereinflüssen: 82.15943 umsatz: 70.88890 cropscience: 61.03360 usa: 60.59338 ebitda: 58.25339 healthcare: 52.58574 ebit: 51.87215 wpb: 49.04000
Topic #4: �: 0.04000 fraport-spezifischer: 0.04000 fremd-: 0.04000 fremdanteile: 0.04000 fremdanteilen: 0.04000 fremdbesitz: 0.04000 fremdbezug: 0.04000 fremdb